# Pokedex DataFrame

the dataframe API in spark is a dynamically typed API which allows pandas/SQL
like requests in a distributed (and possibly noisy) environment.
It compiles to RDD operations on the low level, but the high level is
pretty straightforward.

Most of the time, you will only need to use that API, however, sometimes you will
need statically typed code and some other times (typically on custom aggregations)
you will need to create the low level RDD code yourself. Most of the time though,
you will use DataFrames and DataSets

## Loading the data

In [ ]:
import org.apache.spark.sql.{functions => F}

In [ ]:
// Your spark session has already been created

val df = spark
    .read
    .option("multiline", true)
    .option("header", true)
    .format("csv")
    .load("pokedex_(Update_05.20).csv")

In [ ]:
df.columns

In [ ]:
df
    .select("name")
    .show(5)

## Correcting the types

In [ ]:
df
    .select("_c0")
    .dtypes

In [ ]:
// check the types

In [ ]:
// correct the types using withColumn

## Analyse the data

Which pokemons are the highest?

Which pokemons are the heaviest?

Which pokemons have the biggest BMI (mass / height²)?

In [ ]:
// compute max and select when the value is the max

## Group By and Join

Which pokemons are the heaviest in each generation?

In [ ]:
// compute the groupby aggregations and inner join

# Pokedex Dataset

DataSets are the statically typed equivalent of DataFrames, when working
with datasets, all the types are checked by the Scala compiler, which means
that you will be able to spot mistakes earlier in your process

DataFrames are DataSets with type Row, DataFrame = DataSet\[Row\]
and the type row can take data of any schema

Compute the maximum hp using the typed Dataset API with map and reduce

In [ ]:
// create the case class corresponding to the row types

In [ ]:
// load the data and cast

In [ ]:
df
    .map(_.hp)
    .reduce((x: Integer, y: Integer) => new Integer(Integer.max(x, y)))

# RDD API

The RDD API is low level and most of the time you won't need to use it.
However, in som cases, your Spark jobs might fail because you have a lot of
data and you need to optimise your code if you don't want the cluster to
give up on your jobs, so it is useful to know how to use RDDs.

get the total weight of each generation of pokemons with the RDD API

In [ ]:
import org.apache.spark.rdd.RDD

val dfRDD: RDD[(String, Float)] = df
    .select("generation", "weight_kg")
    .as[(String, Float)]
    .rdd

In [ ]:
dfRDD
    .reduceByKey(_+_)
    .take(10)

# TF IDF IMDB

In [ ]:
val df = spark
    .read
    .option("multiline", true)
    .option("header", true)
    .format("csv")
    .load("IMDB Dataset.csv")
df

## Tokenizing

In [ ]:
import org.apache.spark.ml.feature.Tokenizer

val tkn = new Tokenizer()
    .setInputCol("review")
    .setOutputCol("review_toks")
val tokenized = tkn.transform(df)
tokenized.show()


## Removing stop words

In [ ]:
import org.apache.spark.ml.feature.StopWordsRemover

val englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
val stops = new StopWordsRemover()
    .setStopWords(englishStopWords)
    .setInputCol("review_toks")
    .setOutputCol("review_toks_sw")

val sw = stops.transform(tokenized)

sw.show

## Computing TF-IDF

In [ ]:
import org.apache.spark.ml.feature.{HashingTF, IDF}


val tf = new HashingTF()
    .setInputCol("review_toks_sw")
    .setOutputCol("TFOut")
    .setNumFeatures(10000)

val idf = new IDF()
    .setInputCol("TFOut")
    .setOutputCol("IDFOut")
    .setMinDocFreq(2)

val tfdat = tf.transform(sw)

val tfIdfOut = idf.fit(tfdat)
    .transform(tfdat)

tfIdfOut.select("TFOut", "IDFOut").show